In [1]:
import dask.dataframe as dd
import os

# Указываем шаблон пути для файлов PSX
psx_files_pattern = 'telecom100k/telecom100k/psx/*.csv'

# Читаем файлы, указывая dtype для EndSession, чтобы гарантировать, что он читается как объект (строка)
df_dask = dd.read_csv(psx_files_pattern, dtype={'EndSession': 'object'})

# Если столбец с опечаткой 'Duartion' присутствует, переименовываем его в 'Duration'
if 'Duartion' in df_dask.columns:
    df_dask = df_dask.rename(columns={'Duartion': 'Duration'})

# Вычисляем объединённый DataFrame, преобразуя Dask DataFrame в pandas DataFrame
df_combined = df_dask.compute()

# Сохраняем объединённый датасет в один CSV файл
output_file = 'combined_psx.csv'
df_combined.to_csv(output_file, index=False)

print(f"Объединённый датасет сохранён в '{output_file}'")

Объединённый датасет сохранён в 'combined_psx.csv'


In [2]:
import pandas as pd

df_combined = pd.read_csv('combined_psx.csv')
df_combined

,IdSession,IdPSX,IdSubscriber,StartSession,EndSession,Duration,UpTx,DownTx
0,8969,3,37320,31-12-2023 15:56:30,NaN,29610,1109939500,1420590648
1,15158,3,92681,31-12-2023 14:49:07,NaN,33653,1560226102,2164212384
2,10521,3,36234,31-12-2023 16:02:21,NaN,29259,7389534198,21769968711
3,15161,3,75631,31-12-2023 14:14:09,NaN,35751,724784536,821707317
4,15163,3,34228,31-12-2023 20:25:42,NaN,13458,838533151,913270856
...,...,...,...,...,...,...,...,...
5490480,47743,4,22815,07-01-2024 10:37:48,NaN,48132,1846892025,2368473343
5490481,47702,4,28633,07-01-2024 10:20:54,NaN,49146,2259321089,3259902566
5490482,50135,4,33496,07-01-2024 22:51:24,NaN,4116,281782681,311161751
5490483,47952,4,30895,07-01-2024 11:48:57,NaN,43863,1933356493,2576634716


In [3]:
# 2. Чтение файла с подписчиками (Subscriber)
subscriber_file = 'telecom100k/telecom100k/subscribers.csv'
df_subscriber = pd.read_csv(subscriber_file, dtype={'IdOnPSX': str, 'IdClient': str})

# 3. Приведение типов для объединения
df_combined['IdSubscriber'] = df_combined['IdSubscriber'].astype(str)
df_subscriber['IdOnPSX'] = df_subscriber['IdOnPSX'].astype(str)

# 4. Объединение по полям: PSXStats.IdSubscriber = Subscriber.IdOnPSX
df_merged = pd.merge(df_combined, df_subscriber[['IdClient', 'IdOnPSX']], 
                     left_on='IdSubscriber', right_on='IdOnPSX', how='left')

# Убираем из итогового DataFrame лишний столбец (IdOnPSX) из subscriber
df_merged.drop(columns=['IdOnPSX'], inplace=True)

# 5. Сохраняем результат в новый CSV-файл
output_file = 'combined_psx_with_client.csv'
df_merged.to_csv(output_file, index=False)

print(f"Новый датасет с добавленным IdClient сохранён в '{output_file}'")

Новый датасет с добавленным IdClient сохранён в 'combined_psx_with_client.csv'


In [4]:
df_merged = pd.read_csv('combined_psx_with_client.csv')
df_merged

,IdSession,IdPSX,IdSubscriber,StartSession,EndSession,Duration,UpTx,DownTx,IdClient
0,8969,3,37320,31-12-2023 15:56:30,NaN,29610,1109939500,1420590648,4dc99dc9-7973-472a-abfd-274d3eab5681
1,15158,3,92681,31-12-2023 14:49:07,NaN,33653,1560226102,2164212384,9042aeed-88ec-434c-b99d-71a61b387c64
2,10521,3,36234,31-12-2023 16:02:21,NaN,29259,7389534198,21769968711,1287f0bd-292d-41eb-a69e-817d0edbabe4
3,15161,3,75631,31-12-2023 14:14:09,NaN,35751,724784536,821707317,3df632bb-640d-4d99-ae44-304da7b2ed98
4,15163,3,34228,31-12-2023 20:25:42,NaN,13458,838533151,913270856,3f2908a1-692a-4eb0-a4c5-e1e8c26ab67f
...,...,...,...,...,...,...,...,...,...
5490480,47743,4,22815,07-01-2024 10:37:48,NaN,48132,1846892025,2368473343,c2a78285-e8e9-4baa-a83a-bd5544c39205
5490481,47702,4,28633,07-01-2024 10:20:54,NaN,49146,2259321089,3259902566,33f89d4f-35d0-425b-b32f-d941e1479676
5490482,50135,4,33496,07-01-2024 22:51:24,NaN,4116,281782681,311161751,f312dc49-adfc-4056-a61b-727c57f5e2dc
5490483,47952,4,30895,07-01-2024 11:48:57,NaN,43863,1933356493,2576634716,619f6baa-2529-4551-9187-d2cc022ebab7


In [5]:
# Создаем папку для сохранения файлов, если её нет
output_dir = 'clients'
os.makedirs(output_dir, exist_ok=True)

# Группируем данные по IdClient и сохраняем каждую группу в отдельный CSV-файл
for client_id, group in df_merged.groupby('IdClient'):
    # Сортировка по времени, если требуется (по StartSession)
    group_sorted = group.sort_values('StartSession')
    # Формируем имя файла для клиента
    file_path = os.path.join(output_dir, f'client_{client_id}.csv')
    group_sorted.to_csv(file_path, index=False)

print("Датасет успешно разбит на группы по IdClient, файлы сохранены в папке 'clients'.")

Датасет успешно разбит на группы по IdClient, файлы сохранены в папке 'clients'.


In [6]:
import pandas as pd
import glob
import os

# 1. Считываем исходный файл клиентов (client.parquet)
# Здесь содержится информация о клиенте, включая идентификатор клиента и его тарифный план (IdPlan)
clients_file = 'telecom100k/telecom100k/client.parquet'
df_clients_orig = pd.read_parquet(clients_file)
# Приводим идентификатор клиента к строковому типу и переименовываем для удобства объединения
df_clients_orig['Id'] = df_clients_orig['Id'].astype(str)
df_client_mapping = df_clients_orig[['Id', 'IdPlan']].copy()
df_client_mapping.rename(columns={'Id': 'IdClient'}, inplace=True)
df_client_mapping['IdPlan'] = df_client_mapping['IdPlan'].astype(str)

# 2. Считываем данные тарифных планов (Plan) из JSON
plan_file = 'telecom100k/telecom100k/plan.json'
df_plan = pd.read_json(plan_file)
df_plan['Id'] = df_plan['Id'].astype(str)  # тарифный план хранится в столбце 'Id' в df_plan

# 3. Считываем данные параметров коммутаторов (PSXAttrs) из CSV
psxattrs_file = 'telecom100k/telecom100k/psxattrs.csv'
df_psxattrs = pd.read_csv(psxattrs_file, dtype={'Id': str})
# Если нужно, можно оставить только интересующие столбцы:
# df_psxattrs = df_psxattrs[['Id', 'PSX', 'TransmitUnits', 'Delimiter', 'DateFormat', 'TZ']]

# 4. Находим все файлы клиентов в папке "clients"
client_files = glob.glob(os.path.join('clients', 'client_*.csv'))

# 5. Обрабатываем каждый файл
for file in client_files:
    # Считываем файл клиента. Файл содержит: IdSession, IdPSX, IdSubscriber, StartSession, EndSession, Duration, UpTx, DownTx, IdClient
    df_client = pd.read_csv(file, dtype={'IdClient': str, 'IdPSX': str})
    
    # 5.1. Объединяем с клиентским маппингом для добавления поля IdPlan
    df_client = pd.merge(df_client, df_client_mapping, on='IdClient', how='left')
    
    # 5.2. Объединяем с тарифными планами по полю IdPlan (df_client.IdPlan из маппинга = df_plan.Id)
    df_client = pd.merge(df_client, df_plan, left_on='IdPlan', right_on='Id', how='left', suffixes=('', '_plan'))
    # Если лишний столбец 'Id_plan' появился, его можно удалить (но здесь основное поле тарифного плана – 'IdPlan' уже добавлено)
    
    # 5.3. Объединяем с параметрами коммутаторов (PSXAttrs) по полю IdPSX
    df_client = pd.merge(df_client, df_psxattrs, left_on='IdPSX', right_on='Id', how='left', suffixes=('', '_psxattrs'))
    # При необходимости можно удалить дублирующий столбец, например, 'Id_psxattrs'
    if 'Id_psxattrs' in df_client.columns:
        df_client.drop(columns=['Id_psxattrs'], inplace=True)
    
    # 5.4. Сохраняем обновлённый DataFrame обратно в тот же файл
    df_client.to_csv(file, index=False)

print("Для каждого файла клиента добавлена информация о тарифном плане и PSXAttrs.")

Для каждого файла клиента добавлена информация о тарифном плане и PSXAttrs.


In [14]:
client = pd.read_csv('clients/client_00a1b9b1-f5f2-4ccb-ab9f-c58230202939.csv')
client.drop(columns=['IdPSX', 'IdSubscriber', 'Name', 'Id', 'CreatedAt', 'CreatedAt', 'UpdatedAt', 'ClosedAt'], inplace=True)

client

,IdSession,StartSession,EndSession,Duration,UpTx,DownTx,IdClient,IdPlan,Description,Enabled,Attrs,PSX,TransmitUnits,Delimiter,DateFormat,TZ
0,19341,01-01-2024 09:49:35,NaN,25,184354,254546,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
1,19341,01-01-2024 09:49:35,NaN,41425,222885292,225554002,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
2,19341,01-01-2024 09:49:35,NaN,40825,213214039,231645789,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
3,19341,01-01-2024 09:49:35,NaN,40225,212455760,241743212,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
4,19341,01-01-2024 09:49:35,NaN,39625,66888368,87012546,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,15708,31-12-2023 19:35:52,NaN,30248,68018543,69875063,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
637,15708,31-12-2023 19:35:52,NaN,30848,54639677,74314801,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
638,15708,31-12-2023 19:35:52,NaN,31448,65117292,72839194,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6
639,15708,31-12-2023 19:35:52,NaN,33248,224328908,333695303,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,"Unlimited internet traffic, 500 min calls, int...",True,"I,Unlim,-,500m,ITV",69.0,bytes,",",%d-%m-%Y %H:%M:%S,GMT-6


In [15]:
import pandas as pd

def process_client_data(df):
    # Если в колонке TransmitUnits указано 'bytes', умножаем UpTx и DownTx на 8
    mask = df['TransmitUnits'] == 'bytes'
    df.loc[mask, 'UpTx'] = df.loc[mask, 'UpTx'] * 8
    df.loc[mask, 'DownTx'] = df.loc[mask, 'DownTx'] * 8

    # Функция для расчёта среднего количества пакетов за длительность
    def compute_avg_packets(row):
        if row['Duration'] <= 0:
            return 0
        return (row['UpTx'] + row['DownTx']) / row['Duration']
    
    # Применяем функцию и создаём столбец 'AvgPackets'
    df['AvgPackets'] = df.apply(compute_avg_packets, axis=1)

    # Функция для расчёта среднего исходящего трафика
    def compute_avg_up(row):
        if row['Duration'] <= 0:
            return 0
        return row['UpTx'] / row['Duration']
    
    # Функция для расчёта среднего входящего трафика
    def compute_avg_down(row):
        if row['Duration'] <= 0:
            return 0
        return row['DownTx'] / row['Duration']
    
    # Создаём столбцы 'AvgUp' и 'AvgDown'
    df['AvgUp'] = df.apply(compute_avg_up, axis=1)
    df['AvgDown'] = df.apply(compute_avg_down, axis=1)

    # Функция для извлечения второго значения из строки в колонке Attrs
    def get_second_attr(row):
        # Если Attrs или Delimiter отсутствуют, возвращаем None
        if pd.isna(row['Attrs']) or pd.isna(row['Delimiter']):
            return None
        parts = row['Attrs'].split(row['Delimiter'])
        # Проверяем, что есть как минимум два элемента
        if len(parts) >= 2:
            return parts[1]
        else:
            return None

    # Добавляем новый столбец с вторым значением
    df['EthernetPlan'] = df.apply(get_second_attr, axis=1)
    df = df.drop(columns=['Attrs', 'TransmitUnits', 'PSX', 'Delimiter', 'Description'])
    
    return df

    
# Пример использования:
# df = pd.read_csv('your_data.csv')
# df = process_client_data(df)

In [16]:
import pandas as pd
from datetime import datetime, timezone, timedelta

def convert_to_utc(date_str, date_format, tz_str):
    """
    Парсит дату по date_format и локализует по временной зоне из tz_str (например, 'GMT-6'),
    затем конвертирует время в UTC.
    """
    # Если дата отсутствует, возвращаем None
    if pd.isna(date_str):
        return None
    # Парсинг даты по заданному формату
    dt = datetime.strptime(date_str, date_format)
    # Извлекаем числовое значение временной зоны (например, -6 из 'GMT-6')
    offset = int(tz_str.replace("GMT", ""))
    # Приводим дату к указанной временной зоне
    dt = dt.replace(tzinfo=timezone(timedelta(hours=offset)))
    # Конвертируем дату в UTC
    utc_dt = dt.astimezone(timezone.utc)
    return utc_dt

def process_session_columns(df):
    """
    Функция добавляет в DataFrame два новых столбца:
    - StartSessionUTC: время начала сессии в UTC.
    - EndSessionUTC: время окончания сессии в UTC (если исходное значение NaN, то 0).
    """
    # Обрабатываем StartSession
    df['StartSessionUTC'] = df.apply(
        lambda row: convert_to_utc(row['StartSession'], row['DateFormat'], row['TZ']), axis=1
    )
    # Обрабатываем EndSession: если NaN - возвращаем 0, иначе проводим конвертацию
    df['EndSessionUTC'] = df.apply(
        lambda row: convert_to_utc(row['EndSession'], row['DateFormat'], row['TZ']) 
        if pd.notna(row['EndSession']) else 0, axis=1
    )

    df = df.drop(columns=['StartSession', 'EndSession', 'TZ'])
    return df


# Пример использования:
# df = pd.read_csv('your_data.csv')
# df = process_session_columns(df)
# print(df[['StartSessionUTC', 'EndSessionUTC']].head())

In [17]:
client = process_client_data(client)

In [18]:
client = process_session_columns(client)

In [19]:
client.head(2)

,IdSession,Duration,UpTx,DownTx,IdClient,IdPlan,Enabled,DateFormat,AvgPackets,AvgUp,AvgDown,EthernetPlan,StartSessionUTC,EndSessionUTC
0,19341,25,1474832,2036368,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,True,%d-%m-%Y %H:%M:%S,140448.000000,58993.280000,81454.720000,Unlim,2024-01-01 15:49:35+00:00,0
1,19341,41425,1783082336,1804432016,00a1b9b1-f5f2-4ccb-ab9f-c58230202939,6,True,%d-%m-%Y %H:%M:%S,86602.639759,43043.629113,43559.010646,Unlim,2024-01-01 15:49:35+00:00,0


In [ ]:
client.head(10)